## <center>Логистическая регрессия в задаче классификации текстов</center>

#### <center> Автор материала: Алексей Борисихин

### Описание задачи

Реализована модель multiclass/multilabel классификации текстов, использующая онлайн обучение (стохастический градиентный спуск с логистической функцией потерь). В качестве примера взят датасет вопросов на stackoverflow.com, задачей стоит предсказание тегов вопросов.

Основным преимуществом онлайн обучения является возможность не загружать весь обучающий датасет в память, что позволяет использовать большие датасеты для обучения. Модель обучатеся итеративно по каждому объекту выборки. Соответственно, не составляет труда дообучить модель на новых данных.

Данные доступны для скачивания по ссылке: https://yadi.sk/d/Vvc4JmX13ZdWvk (готовая выборка из 125k объектов).<br>
Используя парсинг страниц сайта stackoverflow.com, можно было бы увеличить размер имеющихся данных. Но, ввиду ограничений сайта на количество запросов, представляется фомировать только небольшие датасеты за раз. Их можно использовать для дообучения модели.

За основу было взято задание курса [OpenDataScience](https://github.com/Yorko/mlcourse_open) (весна 2018). Автор задания - Павел Нестеров.

### Формат входных данных

Входные данные представляют собой файл с вопросами пользователей stackoverflow.com. В каждой строке файла содержится текст вопроса и список тегов, разделенные символом табуляции. Теги в списке разделены пробелом.

### Описание математической основы

#### Логистическая регрессия для двух классов

Рассмотрим логистическую регрессию для двух классов $\{0, 1\}$. Обозначим вектор признаков объекта как $\textbf{x}$. Вероятность принадлежности объекта классу $1$, вспомнив теорему Байеса, можно записать как:

$$
p\left(c = 1 \mid \textbf{x}\right) = 
\dfrac
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right)}
{p\left(\textbf{x}\right)}
$$

Воспользуясь формулой полной вероятности, получаем:

$$
p\left(c = 1 \mid \textbf{x}\right) = 
\dfrac
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right)}
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right) + p\left(\textbf{x} \mid c = 0\right)p\left(c = 0\right)}
$$

Введя параметр:

$$
a = \log \dfrac
{p\left(\textbf{x} \mid c = 1\right)p\left(c = 1\right)}
{p\left(\textbf{x} \mid c = 0\right)p\left(c = 0\right)}
$$

Мы можем наше выражение переписать в виде:

$$
p\left(c = 1 \mid \textbf{x}\right) = \dfrac{1}{1 + e^{-a}} = \sigma\left(a\right)
$$

где $\sigma\left(a\right)$ - обозначение функции логистического сигмоида для скалярного аргумента.

Значение же параметра $a$ мы моделируем линейной функцией от признаков объекта и параметров модели:

$$
a = \sum_{i=0}^M w_i x_i
$$

#### Задача многоклассовой классификации

Обобщим подход до задачи многоклассовой классификации. У нас есть $K$ классов, к которым может принадлежать объект: $\{1, 2, ..., K\}$. Запишем вероятность принадлежности объекта классу $k$:

$$
p\left(c = k \mid \textbf{x}\right) = 
\dfrac
{p\left(\textbf{x} \mid c = k\right)p\left(c = k\right)}
{p\left(\textbf{x}\right)} =
\dfrac
{p\left(\textbf{x} \mid c = k\right)p\left(c = k\right)}
{\sum_{i=1}^Kp\left(\textbf{x} \mid c = i\right)p\left(c = i\right)}
$$

Введем параметр:

$$
z_k = \log p\left(\textbf{x} \mid c=k \right) p\left(c = k\right)
$$

И перепишем наше выражение в виде:

$$
p\left(c = k \mid \textbf{x}\right) =
\dfrac
{e^{z_k}}
{\sum_{i=1}^K e^{z_i}}
= \sigma_k(\textbf{z})
$$

где $\sigma_k$ — $k$-ый компонент функции softmax (обобщение логистической регрессии для многомерного случая) при векторном аргументе. 

Вектор $\sigma$ образует дискретное вероятностное распределение, т.е. $\sum_{i=1}^{K}\sigma_i = 1$.

Значение параметра $z_k$ мы моделируем линейной функцией от признаков объекта (размерности M) и параметров модели для класса $k$:

$$
z_k = \sum_{i=1}^Mw_{ki}x_i
$$

Для моделирования искомого распределения будем использовать [категориальное распределение](https://en.wikipedia.org/wiki/Categorical_distribution). Запишем функцию правдоподобия:

$$
L\left(\theta \mid \textbf{x}, \textbf{y}\right) = 
\prod_{i=1}^{K}p_{i}^{y_{i}} = 
\prod_{i=1}^{K}\sigma_{i}(\textbf{z})^{y_{i}}
$$

Поскольку логарифм положительного аргумента монотонно возрастает на всей области определения, то логарифмирование функции правдоподобия не изменит положение ее максимума. Значит, для удобства мы можем воспользоваться логарифмом функции правдоподобия:

$$
\mathcal{L} = \log L = 
\log \left(\prod_{i=1}^{K}\sigma_{i}(\textbf{z})^{y_{i}}\right) = 
\sum_{i=1}^{K}y_{i}\log \sigma_{i}(\textbf{z}) \rightarrow max
$$

Если домножить на $(-1)$, то получится выражение [кросс-энтропии](https://en.wikipedia.org/wiki/Cross_entropy) для многоклассовой классификации. Правдоподобие мы максимизируем, а кросс-энтропию, соответственно, минимизируем.

$$
H = \left(-\mathcal{L}\right) \rightarrow min
$$

Для этого будем использовать методы градиентного спуска. Необходимо вывести выражение для компонент вектора градиента кросс-энтропии:

$$
\dfrac {\partial H} {\partial w_{km}}
$$

Рассмотрим следующие частные производные:

$$
\dfrac {\partial H} {\partial \sigma_{k}} =
\dfrac {\partial} {\partial \sigma_{k}} \left( -\sum_{i=1}^{K} y_i \log \sigma_i \right) 
= - \dfrac {y_k} {\sigma_k}
$$

<br><br>
$$
\dfrac {\partial \sigma_i} {\partial z_k} = 
\dfrac {\partial} {\partial z_k} \left( \dfrac {e^{z_i}} {\sum_{j=1}^{K} e^{z_j}} \right) =
\left \{ \begin{array}{lcl} 
\dfrac {1} {\left( \sum_{j=1}^{K}e^{z_j} \right)^2 } \left( e^{z_k} \sum_{j=1}^{K}e^{z_j} - e^{z_k} \cdot e^{z_k} \right) &=&
\sigma_k \left( 1 - \sigma_k \right) && (i=k)
\\ 
\dfrac {1} {\left( \sum_{j=1}^{K}e^{z_j} \right)^2 } \left( -e^{z_i} \cdot e^{z_k} \right)
&=& - \sigma_i \sigma_k && (i \neq k)
\end{array} \right.
$$

<br><br>
$$
\dfrac {\partial z_i} {\partial w_{km}} =
\dfrac {\partial} {\partial w_{km}} \left( \sum_{j=1}^{M} w_{ij}x_j \right) = 
\left \{ \begin{array}{lcl} 
x_m && (i=k)
\\
0 && (i \neq k)
\end{array} \right.
$$

Теперь мы можем записать:

$$
\dfrac {\partial H} {\partial z_k} = 
\sum_{i=1}^{K} \dfrac {\partial H} {\partial \sigma_i} \dfrac {\partial \sigma_i} {\partial z_k} =
\dfrac {\partial H} {\partial \sigma_k} \dfrac {\partial \sigma_k} {\partial z_k} +
\sum_{i \neq k} \dfrac {\partial H} {\partial \sigma_i} \dfrac {\partial \sigma_i} {\partial z_k} =
-y_k \left( 1- \sigma_k \right) + \sum_{i \neq k} y_i \sigma_k =
-y_k + \sigma_k \sum_{i} y_i =
\sigma_k - y_k
$$
<br><br>
$$
\dfrac {\partial H} {\partial w_{km}} =
\sum_{i=1}^{K} \dfrac {\partial H} {\partial z_i} \dfrac {\partial z_i} {\partial w_{km}} =
x_m \left( \sigma_k - y_k \right)
$$

#### Задача multilabel классификации

В другой постановке задачи каждый классифицируемый пример может иметь несколько тегов (принадлежать к нескольким классам). В этом случае требуется изменить модель:
- будем считать, что все теги независимы друг от друга, и каждый исход - это логистическая регрессия на два класса (стратегия one-vs-all)
- наличие каждого тега будем моделировать с помощью [распределения Бернулли](https://en.wikipedia.org/wiki/Bernoulli_distribution)

Используя предыдущие выводы по логистической регрессии для двух классов, мы можем записать вероятность наличия тега (принадлежности к классу) следующим образом:

$$
p\left( c=k \mid \textbf{x} \right) = \sigma \left( z_k \right) = \sigma \left( \sum_{i=1}^{M} w_{ki}x_{i} \right)
$$

$$
\sigma(z_k) = \dfrac {1} {1 - e^{-z_k}}
$$

$$
z_k = \sum_{i=1}^Mw_{ki}x_i
$$

Отметим, что каждый тег (класс), как и для случая многоклассовой логрегрессии, имеет свой набор параметров.

Запишем функцию правдоподобия и ее логарифм:

$$
L \left( \theta \mid \textbf{x,y} \right) = \prod_{i=1}^{K} p_i^{y_i}\left( 1-p_i \right)^{1- y_i} =
\prod_{i=1}^{K} \sigma(z_i)^{y_i}\left( 1-\sigma(z_i) \right)^{1- y_i}
$$

<br>
$$
\mathcal{L} = \log L = \log \left( \prod_{i=1}^{K} \sigma(z_i)^{y_i}\left( 1-\sigma(z_i) \right)^{1- y_i} \right) =
\sum_{i=1}^{K} \left( \log \left( \sigma(z_i)^{y_i}\left( 1-\sigma(z_i) \right)^{1- y_i} \right) \right) =
$$

$$
= \sum_{i=1}^{K} \left( y_i \log \sigma \left(z_i \right) + \left( 1-y_i \right) \log \left( 1 - \sigma \left(z_i \right) \right) \right)
$$

И снова, после домножения на $(-1)$, мы получим выражение кросс-энтропии (в этот раз - для $K$ независимых классов). Нашей задачей стоит поиск ее минимума методами градиентного спуска, а для этого необходимо найти выражение для вычисления частных производных по параметрам модели:

$$
H = \left(-\mathcal{L}\right) \rightarrow min
$$
<br><br>
$$
\dfrac {\partial H} {\partial w_{km}}
$$

Рассмотрим частные производные:

$$
\dfrac {\partial H} {\partial \sigma_k} =
\dfrac {\partial} {\partial \sigma_k} \left( (y_k - 1) \log ( 1 - \sigma_k ) - y_k \log \sigma_k \right) =
\dfrac {1 - y_k} {1 - \sigma_k} - \dfrac {y_k} {\sigma_k} =
\dfrac {\sigma_k - y_k} {\sigma_k (1 - \sigma_k)}
$$
<br><br>

$$
\dfrac {\partial \sigma_k} {\partial z_k} =
\dfrac {\partial} {\partial z_k} \left( \dfrac {1} {1 - e^{-z_k}} \right) = 
\dfrac {e^{z_k} \left( e^{z_k} - 1 \right) - e^{z_k} \cdot e^{z_k} } {\left( e^{z_k} - 1 \right)^2} =
\sigma_k - \sigma_k^2 = 
\sigma_k (1 - \sigma_k)
$$
<br><br>

$$
\dfrac {\partial z_k} {\partial w_{km}} =
\dfrac {\partial} {\partial w_{km}} \left( \sum_{i=1}^{M} w_{ki} x_i \right) = x_m
$$

Теперь мы можем записать:

$$
\dfrac {\partial H} {\partial w_{km}} =
\dfrac {\partial H} {\partial \sigma_k} \dfrac {\partial \sigma_k} {\partial z_k} 
\dfrac {\partial z_k} {\partial w_{km}} =
x_m \left( \sigma_k - y_k \right)
$$

#### Регуляризация

Для борьбы с переобучением модели будем использовать ElasticNet-регуляризацию. Это комбинация L1 (провоцирует на обнуление весов у малоинформативных признаков) и L2 (штрафует за большое по модулю значение весов) типов регуляризации. Запишем слагаемые в общей функции потерь, соответствующие L1 и L2 регуляризациям:

$$
L_1 = \sum_{i=1}^{K} \sum_{j=1}^{M} \mid w_{ij} \mid
$$

$$
L_2 = \sum_{i=1}^{K} \sum_{j=1}^{M} w_{ij}^2
$$

Введем параметры $\lambda$ (коэффициент регуляризации) и $\gamma$ (соотношение между двумя типами регуляризации). Тогда функция потерь примет вид:

$$
H_{reg} = H + \lambda \left( \gamma \sum_{i=1}^{K} \sum_{j=1}^{M} w_{ij}^2 + 
(1 - \gamma) \sum_{i=1}^{K} \sum_{j=1}^{M} \mid w_{ij} \mid \right)
$$

Частная производная компонента регуляризации тогда запишется как:

$$
\dfrac {\partial} {\partial w_{km}} \left( \lambda \left( \gamma \sum_{i=1}^{K} \sum_{j=1}^{M} w_{ij}^2 + 
(1 - \gamma) \sum_{i=1}^{K} \sum_{j=1}^{M} \mid w_{ij} \mid \right) \right) =
\lambda \left( 2 \gamma w_{km} + (1 - \gamma)sign(w_{km}) \right)
$$

И, наконец, общий вид частной производной функции потерь с регуляризацией будет выглядеть как:

$$
\dfrac {\partial H} {\partial w_{km}} = 
x_m \left( \sigma_k - y_k \right) + \lambda \left( 2 \gamma w_{km} + (1 - \gamma)sign(w_{km}) \right)
$$

### Метрика качества

В качестве метрики качества классификации предлагается использовать [коэффициент Жаккара](https://en.wikipedia.org/wiki/Jaccard_index). Для конкретного объекта классификации коэффициент определяется как:

$$
J(T_p, T_t) = \dfrac {\mid T_p \cap T_t \mid} {\mid T_p \cup T_t \mid}
$$

где $T_p$ - определенное моделью множество тегов объекта, а $T_е$ - фактическое множество тегов объекта. 

Метрика принимает значения на интервале $[0, 1]$, полное соответствие оценивается как $1$. Результирующую метрику на всей тестовой выборке будем определять как среднее по объектам.

### Файлы проекта

- <b>olr_class.py</b>

    Объявление и реализация класса модели.<br><br>
    
- <b>stackoverflow_scrapping.ipynb</b>

    Notebook, выполняющий web scraping (парсинг данных) с сайта stackoverflow.com и формирующий датасет.<br><br>

- <b>data_preprocess.ipynb</b>

    Notebook, подготавливающий входные данные для модели.<br>
    Сначала отбираются top-n тегов по всей выборке. Далее выборка фильтруется в соответствии с этими тегами (записи, не содержащие ни одного тега из top-n, не включаются в выборку; для остальных из всего списка тегов оставляются только входящие в top-n). После этого выборка разбивается в соответствии с заданной пропорцией на тренировочную и тестовую.Теги вынесены в отдельный файл.<br><br>
    
- <b>estimator_test.ipynb</b>

    Notebook, содержащий тестирование класса модели.<br><br>
    
- <b>settings.json</b>

    Найстройки проекта в формате json. Описания полей:
    - data_dir - путь к каталогу данных
    - data_file - имя файла датасета
    - top_tags_count - количество top-n тегов, с которыми будет работать модель
    - top_tags_file - имя создаваемого файла с top-n тегами
    - filtered_tmp_file - имя создаваемого файла отфильтрованной выборки (в процессе удаляется)
    - train_size - относительный размер тренировочного датасета
    - train_file - имя создаваемого файла тренировочного датасета
    - train_labels_file - имя создаваемого файла с тегами тренировочного датасета
    - test_file - имя создаваемого файла тестового датасета
    - test_labels_file - имя создаваемого файла с тегами тестового датасета
    - additional_data_dir - путь к каталогу дополнительных данных (scraping)
    - additional_data_file - шаблон имен файлов дополнительных датасетов
    - objects_to_scrap - количество объектов, которые необходимо получить с сайта

### Тестирование модели

Для тестирования модели использовался датасет из 125 тысяч вопросов с stackoverflow.com с проставленными тегами. Выборка была разбита на обучающую и валидационную, соотношение - 0.8 (100 тысяч примеров для обучения и 25 тысяч - для валидации).<br><br>

Тестирование модели проводилось по трем схемам обучения. После каждого прогона по обучающему датасету сохранялись значения функции потерь и оценки качества на обучающей и валидационной выборках. Далее строились графики - скользящее среднее функции потерь (окно в 20000 примеров) и метрики качества. Неоговоренные параметры модели оставлялись со значениями по умолчанию.

- <b>стратегия обучения - one-vs-rest, коэффициент регуляризации $\lambda$=0.0002, порог предсказания класса - 0.5</b>. Выполняется первичный проход по датасету с формированием словаря, затем словарь фильтруется до топ-25000 популярных слов и выполянется еще один проход по датасету, затем фильтрация топ-10000 с еще одним проходом, и далее 2 прохода по обучающей выборке.
<img src='./img/ovr_reg.png'>
Можно наблюдать повышение качества классификации после каждой фильтрации словаря и последующим дообучением. Дополнительные проходы после фильтраций снижают качество классификации. Значения функции потерь для первого прохода максимальны, для 3-го и 4-го проходов имеют значительные всплески в районе середины обучающей выборки, чем отличаются по форме графика скользящего среднего от остальных проходов.

- <b>стратегия обучения - multinomial, коэффициент регуляризации $\lambda$=0.0002, порог предсказания класса - 0.2</b>. Выполняется первичный проход по датасету с формированием словаря, затем словарь фильтруется до топ-25000 популярных слов и выполянется еще один проход по датасету, затем фильтрация топ-10000 с еще одним проходом, и далее 2 прохода по обучающей выборке.
<img src='./img/mltnom_reg.png'>
В целом графики скользящего среднего функций потерь имеют более шумный характер. Можно наблюдать повышение качества классификации после первой фильтрации словаря и дообучением после второй фильтрации. Последний проход снижает качество классификации. Значения функции потерь для первого прохода максимальны, для остальных проходов плотно лежат в одной области. Формы графиков для всех проходов идентичны.

- <b>стратегия обучения - one-vs-rest, коэффициент регуляризации $\lambda$=0, порог предсказания класса - 0.8</b>. Выполняется первичный проход по датасету с формированием словаря, затем следует 4 прохода по обучающей выборке. Частотный словарь не сохраняется, что немнго увеличивает скорость первого прохода по датасету.
<img src='./img/ovr_nonreg.png'>
Графики функции потерь имеют самый сглаженный характер из всех схем обучения. Для обучающей выборки метрика качества росла с каждым проходом, для валидационной - на последнем проходе качество снизилось, что может говорить о переобучении (модель в данном случае не использовала регуляризацию). Значения функции потерь для первого прохода максимальны, для 3 и 4 проходов опять наблюдается всплеск потерь, в остальном формы графиков можно назвать идентичными. Модель с этими параметрами достигла максимальной оценки качества на валидационной выборке, но отличия незначительны относительно подобной модели, но с регуляризацией.